# Solar System Objects in a field of view

The purpose of the tutorial is to retrieve data from an API (Application Programming Interface).
Once you will have done it once, you will see that 

1. it's easy!
2. many Web services in astronomy can be accessed by adapting the code here below. 


To illustrate the query, we will search for all Solar System objects in a field of view. In the
<a href="https://www.ivoa.net/">Virtual Observatory</a> terminology, this is called a
<kbd><a href="https://www.ivoa.net/documents/latest/ConeSearch.html">cone-search</a></kbd>.

We must specify the direction (**coordinates**) at which we are pointing at, and a **radius** around to search for targets. As Solar System objects constantly move, the **epoch** of observation is also required. The general philosophy is to provide all the parameters as keywords (in a <kbd>param=value</kbd> fashion) to the URL of the service, which returns the answer. 

<kbd><a href="https://www.ivoa.net/documents/latest/ConeSearch.html">cone-search</a></kbd> are core services in the
<a href="https://www.ivoa.net/">Virtual Observatory</a> for fixed objects (stars, galaxies, ...). They are, however, only a few for Solar System:

- **MPChecker** of the [Minor Planet Center](https://minorplanetcenter.net), available in [Web form](https://minorplanetcenter.net/cgi-bin/checkmp.cgi) only
- **SkyBoT** of the [IMCCE](https://www.imcce.fr/), available in [Aladin](https://aladin.cds.unistra.fr/) and with an [API](https://ssp.imcce.fr/webservices/skybot/)
- **Small-Body Identification Tool** of the [JPL Solar System Dynamics](https://ssd.jpl.nasa.gov/), available in [Web form](https://ssd.jpl.nasa.gov/tools/sb_ident.html) and [API](https://ssd-api.jpl.nasa.gov/doc/sb_ident.html)

We will use **SkyBoT** here.

In [ ]:
import io
import requests
from astropy.coordinates import Angle

import pandas as pd

import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord

## Define the parameters of the query

All services with an API have a documentation explaining how to use them. For a complete description of the parameters, we highly recommend to look at the documentation:
[https://ssp.imcce.fr/webservices/skybot/api/conesearch/](https://ssp.imcce.fr/webservices/skybot/api/conesearch/)

In [ ]:
# When we observe
epoch = "2022-06-21T00:00:00"

# Where we observe
ra = "07h08m00"
dec = "+26d34m00"
radius = 0.25 # degrees

# Where are we observing from (observatory IAU code, 500=geocenter)
observer = "500"

# Filename to store results
filename = "my_conesearch.csv"

## Build the query parameters

This step will be very similar for all APIs you query: you specify the URL of the service and create a `dict` containing the parameter-value pairs you want to pass to the request.

In [ ]:
# Service URL
url = "https://ssp.imcce.fr/webservices/skybot/api/conesearch.php?"

# Query parameters
# The request package that we will use will create the full URL for us from the dictionary below
params = {
    "-ep": epoch,
    "-ra": Angle(ra).degree,
    "-dec": Angle(dec).degree,
    "-rd": radius,
    "-mime": "json",
    "-output": "all",
    "-loc": observer,
    "-tscale": "UTC",
    '-objFilter': '110'
}

## Querying the service

The critical part in the code below is the `requests.post` line, which executes an HTTP POST request, used to send data to a server. We wrap it in a try-except block because sometimes, servers do not respond in time, and the request times out with an error.

In [ ]:
try:
    r = requests.post(url, params=params, timeout=2000)
except requests.exceptions.ReadTimeout:
    print("Request timed out")

That's it! Easy isn't it?

The variable <kbd>r</kbd> contains the results of the <kbd><a href="https://www.ivoa.net/documents/latest/ConeSearch.html">cone-search</a></kbd>. We can extract its content and write it to disk for later usage if needed (*disk access generally slows codes*). 

## Reading the results

The `io.BytesIO` function is a neat little trick to create a "file-like" object in `python` that we can pass to functions like `read_json`. Alternatively, we can write the file to disk and pass the filename to `read_json`.

In [ ]:
# Read results in a pandas DataFrame
data = pd.read_json(io.BytesIO(r.content))

In [ ]:
# Inspect the results, brightest first
data.sort_values(by="VMag (mag)").head()

## Writing the results to disk

In [ ]:
# Write results
data.to_csv(filename, index=False)

## Plot the results

In [ ]:
# Convert RA/Dec from hours/degrees to degrees
coords = SkyCoord(data["RA (deg)"], data["DEC (deg)"], unit=("hour", "deg"))

data["RA"] = coords.ra.degree
data["DEC"] = coords.dec.degree

In [ ]:
# Create a figure and axe objects
fig, ax = plt.subplots()

# Plot the asteroids (size is inversely proportional to magnitude)
ax.scatter(data["RA"], data["DEC"], s=80 - 3 * data["VMag (mag)"], color="C0")

# Show the arrows of motion
scale = 3000  # Visual scaling factor for the arrows
for i, row in data.iterrows():
    ax.arrow(
        row["RA"],
        row["DEC"],
        row["dRA (arcsec/h)"] / scale,
        row["dDEC (arcsec/h)"] / scale,
        color="C0",
    )

# Set axes
ax.set_aspect("equal")
ax.set_xlabel("Right Ascencion (deg)")
ax.set_ylabel("Declination (deg)")
ax.xaxis.set_inverted(True)

plt.show()

# What about the stars?

We have queried a <kbd><a href="https://www.ivoa.net/documents/latest/ConeSearch.html">cone-search</a></kbd> API for the Solar System.
Depending on the data provider, they may also provide some library/packages (in java, python, ...) to build the query for you.

In that respect, the <kbd>[astroquery](https://astroquery.readthedocs.io/en/latest/index.html)</kbd> package (affiliated to 
<kbd>[astropy](https://www.astropy.org/)</kbd>) offers a common interface to **many** services
([CADC](https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/),
[CDS](https://cds.unistra.fr/),
[ESA](https://www.esa.int/),
[ESO](https://www.eso.org/),
[MAST](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html),
[MPC](https://minorplanetcenter.net/),
[NASA Ads](https://ui.adsabs.harvard.edu/),
[IMCCE](https://www.imcce.fr/),
...
)

So let's use the 
<kbd>[Gaia](https://astroquery.readthedocs.io/en/latest/gaia/gaia.html)</kbd> module of 
<kbd>[astroquery](https://astroquery.readthedocs.io/en/latest/index.html)</kbd> to retrieve 
[Gaia](https://gea.esac.esa.int/archive/) stars in the same field, and overplot them (this would be the way to query stars for astrometry and photometry calibration for instance).

In [ ]:
import astropy.units as u
from astroquery.gaia import Gaia

# Define the cone-search center (as astropy SkyCoord object)
coord = SkyCoord(ra=ra, dec=dec, unit=(u.hour, u.degree), frame="icrs")

# Query Gaia DR3
# Here the columns keyword is optional, but it is good practice to specify only the columns you need
Gaia.ROW_LIMIT = -1  # To get all entries
j = Gaia.cone_search_async(
    coord,
    radius=u.Quantity(radius, u.deg),
    columns=["ra", "dec", "phot_g_mean_mag", "phot_bp_mean_mag", "phot_rp_mean_mag"],
)

# Get results
catalog = j.get_results()

In [ ]:
# Inspect the retrieved
catalog[:5]

In [ ]:
fig, ax = plt.subplots()

# Plot Gaia stars
ax.scatter(
    catalog["ra"],
    catalog["dec"],
    s=20 - catalog["phot_g_mean_mag"],
    color="grey",
    label="Stars (Gaia)",
)

# Plot the asteroids (size is inversely proportional to magnitude)
ax.scatter(
    data["RA"], data["DEC"], s=80 - 3 * data["VMag (mag)"], color="C0", label="SSOs (SkyBoT)"
)

# Show the arrows of motion
scale = 3000  # Visual scaling factor for the arrows
for i, row in data.iterrows():
    ax.arrow(
        row["RA"],
        row["DEC"],
        row["dRA (arcsec/h)"] / scale,
        row["dDEC (arcsec/h)"] / scale,
        color="C0",
    )

# Set axes
ax.set_aspect("equal")
ax.set_xlabel("Right Ascencion (deg)")
ax.set_ylabel("Declination (deg)")
ax.legend(loc="upper right")
ax.xaxis.set_inverted(True)

plt.show()

Here it is. We have easily drawn a portion of the sky, with both the stars and the Solar System objects!
So, whenever you plan to access data, check out

- if there is an API, then <kbd>[requests](https://pypi.org/project/requests/)</kbd> is your friend
- check if <kbd>[astroquery](https://astroquery.readthedocs.io/en/latest/index.html)</kbd> has already implemented a solution for you

Remember that services are made by ourselves for ourselves, as a community. Consider providing APIs to the data sets and services you produce.
(*side note: the [Vizier](https://vizier.cds.unistra.fr/viz-bin/VizieR) service of the [CDS](https://cds.unistra.fr/) is a extremely simple way to publish your data upon publication*).

**Want to practice?**

- Save the Gaia catalog to a local file.
- Color-code the Gaia star based on their intrinsic colors (B<sub>p</sub>-R<sub>p</sub>) on the plot.
- Search for Solar System objects around the coordinates (RA,Dec) = (02 13 25, +12 06 43) on 2024-01-01, with a radius of one degree. 
- Try the exercises proposed in the notebook `3.2.3 Exercises`
- Get experience with the exercises proposed in the notebook `3.3.3 Advanced exercises`